<a href="https://colab.research.google.com/github/ankokovin/MyMLNotebooks/blob/master/MyCETINCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CETIN Calculator

## 1. Оценка функционального размера

In [120]:
#@title Ввод C,E,T,I и N { run: "auto", display-mode: "form" }
C = 10 #@param {type:"integer"}
E = 10 #@param {type:"integer"}
T = 10 #@param {type:"integer"}
I = 10 #@param {type:"integer"}
N = 10 #@param {type:"integer"}

CETIN = [C, E, T, I, N]
print(CETIN)

[10, 10, 10, 10, 10]


## 2. Расчёт базовой трудоёмкости

|#|Наименование процесса     |C        |E        |T         |I         |N       |
|-|------------------------------------------------|----------|---------|----------|---------|---------|
|1|Бизнес-моделирование        |32.12|28.33|0         |14.15|0        |
|2|Управление требованиями |58.03|28.04|0         |20.32|0        |
|3|Проектирование                     |45.42|61.75|31.35|37.52|24.02|
|4|Реализация                              |31.57|81.51|50.72|36.11|0         |
|5|Тестирование                          |88.96|0       |0         |0       |0         |
|6|Развёртывание                       |8.96 |0        |0         |0       |23.74 |


In [0]:
norm = [
[32.12, 28.33, 0    , 14.15, 0    ],
[58.03, 28.04, 0    , 20.32, 0    ],
[45.42, 61.75, 31.35, 37.52, 24.02],
[31.57, 81.51, 50.72, 36.11, 0    ],
[88.96, 0    , 0    , 0    , 0    ],
[8.96 , 0    , 0    , 0    , 23.74]
]

$S_i = [C*S_i(C) + E*N_i(E) + T * N_i(T) + I*N_i(I) + N * S_i(N)]/165$

или по-русски (по матричному)

$S =  \begin{pmatrix}
  32.12&& 28.33&& 0      && 14.15&& 0        \\
  58.03&& 28.04&& 0      && 20.32&& 0        \\
  45.42&& 61.75&& 31.35&& 37.52&& 24.02 \\
  31.57&& 81.51&& 50.72&& 36.11&& 0        \\
  88.96&& 0      && 0       && 0      && 0        \\
  8.96  && 0      && 0       && 0      && 23.74
 \end{pmatrix} *
 \begin{pmatrix}
  C  \\
  E  \\
  T \\
  I \\
  N
 \end{pmatrix}*\frac{1}{165}$
 $S = norm*CETIN^T*\frac{1}{165}$

In [122]:
import numpy as np

S = np.transpose( np.dot(np.matrix(norm), np.transpose( np.matrix(CETIN))))/165
print(S)

[[ 4.52121212  6.44787879 12.12484848 12.11575758  5.39151515  1.98181818]]


## 3. Определение поправочных коэффициентов

In [0]:
#@title Определение частных поправочных коэффициентов { run: "auto", display-mode: "form" }

#@markdown ## Внутренние факторы

#@markdown ### K1 Режим эксплуатации ИС
#@markdown Определяется в зависимости от конкретных технологий или типов обработки, принятых в системе программного обеспечения. Принимает следующие значения:
#@markdown - 1 - обработка данных в режиме разделения времени
#@markdown - 1,04 - параллельная обработка данных
#@markdown - 1,05 - обработка данных в режиме реального времени
#@markdown - 1,07 - совмещенная обработка данных
K1 = 1 #@param ["1", "1.04", "1.05", "1.07"] {type:"raw"}

#@markdown ### K2 Масштаб ИС
#@markdown Масштаб может быть определен количеством одновременно работающих пользователей. Принимает следующие значения: Принимает следующие значения:
#@markdown - 0,95  малые ИС (до 10 пользователей с непродолжительным ЖЦ)
#@markdown - 1 - средние ИС (от 11 до 100 пользователей с длительным ЖЦ с возможностью роста до крупных систем)
#@markdown - 1,05 - крупные ИС (от 101 до 1000 пользователей с длительным ЖЦ и миграцией унаследованных систем)
#@markdown - 1,08 - сверхбольшие (свыше 1000 пользователей)
K2 = 1 #@param ["0.95", "1", "1.05", "1.08"] {type:"raw"}

#@markdown ### K3 Стабильность ИС 
#@markdown Определяется в зависимости от ее внутренних эволюционных аспектов или стабильности в процессе сопровождения. Принимает следующие значения:
#@markdown - 1,1 - постоянное внесение изменений
#@markdown - 1 - дискретное внесение изменений
#@markdown - 0,95 - маловероятное внесение изменений
K3 = 1 #@param ["1.1","1","0.95"]{type:"raw"}

#@markdown ### K4 Защита от несанкционированного доступа
#@markdown Предотвращение или существенное затруднение несанкционированного доступа. Принимает следующие значения:
#@markdown - 1,05 - сильная 
#@markdown - 1 - средняя
#@markdown - 0,98 - слабая
K4 = 1 #@param ["1.05", "1", "0.98"]{type:"raw"}

#@markdown ### K5 Защита программ и данных (на уровне ОС, на уровне сетевого ПО, на уровне СУБД)
#@markdown - 1,03 - сильная
#@markdown - 1 - средняя
#@markdown - 0,97 - слабая
K5 = 1 #@param ["1.03", "1", "0.97"]{type:"raw"}

#@markdown ### K6 Контрольный след операций
#@markdown Возможность фиксации несанкционированных изменений в системе:
#@markdown - 1 - не имеется 
#@markdown - 1,08 - выборочное отслеживание 
#@markdown - 1,13 - полное отслеживание 
K6 = 1 #@param ["1", "1.08", "1.13"]{type:"raw"}

#@markdown ### К7 Отказоустойчивость
#@markdown Свойство системы непрерывно сохранять работоспособное состояние в течении некоторого времени. Принимает следующие значения:
#@markdown - 1,15 - Высокая 
#@markdown - 1 - Средняя
#@markdown - 0,92 - Низкая
K7 = 1 #@param ["1.15", "1", "0.92"]{type:"raw"}

#@markdown ### K8 Восстанавливаемость 
#@markdown Среднее время восстановления работоспособности после отказа. Принимает следующие значения:
#@markdown - 1,12 - высокая 
#@markdown - 1 - средняя 
#@markdown - 0,98 - низкая 
K8 = 1 #@param ["1.12", "1", "0.98"]{type:"raw"}

#@markdown ### K9 Длительность обработки (время отклика)
#@markdown Быстрота реакции системы на входные воздействия. Принимает следующие значения:
#@markdown - 1,21 - быстрая 
#@markdown - 1 - умеренная
#@markdown - 0,92 - медленная
K9 = 1 #@param ["1.21", "1", "0.92"]{type:"raw"}

#@markdown ### K10 Исходный язык разработки ИС
#@markdown Определяется в зависимости от типа исходного языка, используемого при разработке ИС. Принимает следующие значения:
#@markdown - 1,08 - традиционный (Кобол, Фортран и т.д.)
#@markdown - 1,09 - процедурный (Си или эквивалентный)
#@markdown - 1,07 - функциональный (Лисп или эквивалентный)
#@markdown - 1 -объектноориентированный (Си++ или эквивалентный)
K10 = 1 #@param ["1.08", "1.09", "1.07", "1"]{type:"raw"}

#@markdown ## Факторы среды

#@markdown ### K11 Класс пользователя
#@markdown Определяется в зависимости от уровня мастерства или характеристик определенного класса пользователей. Пользователем может быть система, являющаяся внешней по отношении к рассматриваемой системе. Принимает следующие значения:
#@markdown - 1,12 - начинающий 
#@markdwon - 1,07 - средний 
#@markdown - 1 - специалист (эксперт)
#@markdown - 1,14 - случайный 
#@markdown - 1,06 - другая ИС (ПО) 
#@markdown - 1,09 - технические средства 
K11 = 1 #@param ["1.12", "1.07", "1", "1.14", "1.06", "1.09"]{type:"raw"}

#@markdown ### K12 Требования к центральному обрабатывающему устройству (процессору)
#@markdown Определяются требованиями к тактовой частоте процессора (скорости процессора). Принимает следующие значения: 
#@markdown - 0,99 - высокая 
#@markdown - 1 - средняя
K12 = 1 #@param ["0.99", "1"]{type:"raw"}

#@markdown ### K13 Требования к оперативной (основной) памяти
#@markdown ИС должна быть идентифицировано по требованиям, предъявляемым к оперативной памяти (объем, быстродействие). Принимает следующие значения:
#@markdown - 1 - большая
#@markdown - 1,04 - малая
K13 = 1 #@param ["1", "1.04"]{type:"raw"}

#@markdown ### K14 Требования к внешней памяти
#@markdown ИС должна быть идентифицировано по требованиям, предъявляемым к внешней памяти (объем, быстродействие). Принимает следующие значения:
#@markdown - 1 - большая
#@markdwon - 1,01 - малая 
K14 = 1 #@param ["1", "1.01"]{type:"raw"}

#@markdown ### K15 Требования к локальной вычислительной сети
#@markdown ИС должна быть идентифицировано по требованиям, предъявляемым к ЛВС (пропускная способность, зашита информации в сети). Принимает следующие значения:
#@markdown - 1 - высокие требования
#@markdown - 1,02 - средние требования 
K15 = 1 #@param ["1", "1.02"]{type:"raw"}

#@markdown ### K16 Критичность ИС
#@markdown Определяется в зависимости от уровня целостности продукции, с учетом конкретной методологии оценки. Принимает следующие значения:
#@markdown - 1,18 - человеческая жизнь
#@markdown - 1,16 - национальная безопасность
#@markdown - 1,13 - социальный хаос и паника 
#@markdown - 1 - организационная безопасность
K16 = 1 #@param ["1.18", "1.16", "1.13", "1"]{type:"raw"}

#@markdown ### K17 Готовность
#@markdown Определяется в зависимости от типа имеющегося в наличии ППО. Принимает следующие значения:
#@markdown - 0,99 - наличие в готовом виде (есть альтернативные продукты)
#@markdown - 1 - общедоступная (известная методика)
#@markdown - 1,11 - заказное (методика заказчика специфическая) 
#@markdown - 1,09 - запатентованное (методика разработчика)
K17 = 1 #@param ["0.99", "1", "1.11","1.09"]{type:"raw"}

#@markdown ## Факторы данных

#@markdown ### K18 Представление данных
#@markdown Определяется в зависимости от элементов, типов и структур данных. Принимает следующие значения:
#@markdown - 1 - реляционный
#@markdown - 1 - индексируемый(иерархический)
#@markdown - 1,08 - сетевой
#@markdown - 1,09 - объектный
#@markdown - 0,95 - форматированный файл
K18 = 1 #@param ["1", "1.08", "1.09", "0.95"]{type:"raw"}


In [124]:
k = [-1, K1, K2, K3, K4, K5, K6, K7, K8, K9, K10, 
     K11, K12, K13, K14, K15, K16, K17, K18]

k_kp = [
    [11, 16, 17],
    [1,2,4,5,6,7,8,9,16,17,18],
    [1,2,4,5,6,7,8,9,11,12,13,14,15,16,17,18],
    [1,2,4,5,6,7,8,9,10,12,13,14,15,16,17,18],
    [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],
    [1,2,11,16,18]
]
kp = []
for l in k_kp:
    temp = 1
    for i in l:
        temp *= k[i]
    kp.append(temp)
    
print(kp)

[1, 1, 1, 1, 1, 1]


## 4. Расчет трудоемкости разработки ПО с учетом поправочных коэффициентов

$S=КП_1*S_1+КП_2 * S_2+KП_3 * S_3+КП_4 * S_4+КП_5 *S_5+КП_6*S_6$

In [125]:
S = np.sum(np.multiply(kp, S))
print('Трудоёмкость в человеко/месяцах',S)

Трудоёмкость в человеко/месяцах 42.583030303030306


# 5. Оценка сроков разработки ПО

In [126]:
times = [
(5, 30),
(10, 80),
(17 , 140),
(26 , 210),
(37 , 280),
(50 , 340),
(65 , 400),
(80 , 450),
(100 , 500),
(120 , 550),
(140 , 610),
(160 , 670),
(180 , 720),
(200 , 770),
(230 , 820),
(260 , 870),
(290 , 930),
(330 , 990),
(370 , 1040),
(420 , 1090),
(470 , 1150),
(530 , 1200),
(600 , 1250),
(670 , 1300),
(750 , 1350),
(830 , 1400),
(900 , 1450),
(970 , 1500),
(1150 , 1550),
(1230 , 1600),
(1310 , 1660),
(1390 , 1720),
(1470 , 1780),
(1520 , 1840),
(1570 , 1900),
(1620 , 1960),
(1680 , 2020)
]

step = 40

answer_l = -1
if S >= times[-1][0] + step:
    answer_l = (S - times[-1][0])//step + len(times) + 1
else:
    for i, item in enumerate(times):
        if (item[0] >= S):
            answer_l = max(1,i)
            break

answer_r = -1
if S>= times[-1][1]:
    answer_l = (S - times[-1][1])//step + len(times) + 1
else:
    for i, item in enumerate(times):
        if (item[1]>S):
            answer_r = i + 1
            break

t = (answer_l + answer_r)/2

print('Длительность (месяцы)',t, ' (',answer_r,'-',answer_l,')')
        
    

Длительность (месяцы) 3.5  ( 2 - 5 )


In [128]:
#@title Корректировка трудоемкости при уменьшении срока разработки (при необходимости) { vertical-output: true }
X_percent = 5 #@param {type:"slider", min:0, max:100, step:1}
L = 0.75 #@param {type:"number"}
S_cor = S*X_percent/100*L
print(S_cor)

1.5968636363636366


# 6. Оценка стоимости ПО

In [136]:
#@title Расчёты { run: "auto", vertical-output: true }
avg_wages_this_year = 50000 #@param {type:"integer"}
avg_inflation = 3 #@param {type:"slider", min:0, max:200, step:1}
soc_tax = 13 #@param {type:"slider", min:0, max:100, step:1}
overhead = 71 #@param {type:"slider", min:0, max:100, step:1}
period_cost = 48 #@param {type:"slider", min:0, max:100, step:1}
profitability = 10 #@param {type:"slider", min:0, max:100, step:1}
vat = 20 #@param {type:"slider", min:0, max:100, step:1}
import math

r_sr = math.ceil(t/12)
print(r_sr)

avg_wage = [avg_wages_this_year]

for i in range(r_sr-1):
    temp = avg_wage[-1]*(1+avg_inflation/100)
    avg_wage.append(temp)
print(avg_wage)    
avg_costs = []

for i in range(r_sr):
    temp = avg_wage[i]
    temp *= 1 + (0.9*soc_tax + overhead + period_cost)/100
    temp *= 1 + profitability / 100
    temp += 1 + vat / 100
    avg_costs.append(temp)
print(avg_costs)
Costs = sum(avg_costs)*S**r_sr/r_sr**r_sr
print(Costs)

1
[50000]
[126886.20000000001]
5403198.899636365


# 7. Оценка стоимости сопровождения

In [138]:
#@title Расчёты { run: "auto", vertical-output: true }
N_coef = 15 #@param {type:"slider", min:0, max:100, step:1}
s_Costs = Costs * N_coef / 100 * k[2] * k[3] * k[16]
print(s_Costs)

810479.8349454547
